<a href="https://colab.research.google.com/github/pkhlingam09/Fashion-Search-AI/blob/Find_Nulls/Fashion_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Dataset Link (Kaggle)
# https://www.kaggle.com/datasets/djagatiya/myntra-fashion-product-dataset

In [2]:
# import os

# def find_file(root_dir, file_name):
#     for dirpath, dirnames, filenames in os.walk(root_dir):
#         if file_name in filenames:
#             return os.path.join(dirpath, file_name)
#     return None

# file_path = find_file("/", "Fashion Dataset v2.csv") #Searches from the root directory

# if file_path:
#   print(file_path)
# else:
#   print("File not found")

In [3]:
#############################################################################################################################################################################################################################
###################################################################################################   MYNTRA ASSIGNMENT   ###################################################################################################

In [4]:
file_path = "/content/drive/MyDrive/Fashion_Search_AI/"

In [5]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
def read_dataset():
  !pip install -q kaggle chromadb
  files.upload()
  !mkdir -p ~/.kaggle
  !cp kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json
  !cat ~/.kaggle/kaggle.json
  !kaggle datasets download "djagatiya/myntra-fashion-product-dataset" -p file_path
  !unzip file_path+"myntra-fashion-product-dataset.zip" -d file_path

In [7]:
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.4 MB/s eta 0:00:

In [8]:
import os
import pathlib

import string
import re
import ast
import json
import numpy as np
import pandas as pd

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

from openai import OpenAI
from sentence_transformers import CrossEncoder, util

In [9]:
# pd.set_option('display.max_rows', 200)
# pd.set_option('display.max_columns', 200)

In [10]:
gpt_model = "gpt-4o-mini"
moderator = "omni-moderation-latest"

In [11]:
df = pd.read_csv(file_path + "Fashion Dataset v2.csv")

In [12]:
df.head(5)

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [13]:
df.shape

(14214, 11)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14214 entries, 0 to 14213
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   p_id          14214 non-null  int64  
 1   name          14214 non-null  object 
 2   products      14214 non-null  object 
 3   price         14214 non-null  float64
 4   colour        14214 non-null  object 
 5   brand         14214 non-null  object 
 6   img           14214 non-null  object 
 7   ratingCount   6530 non-null   float64
 8   avg_rating    6530 non-null   float64
 9   description   14214 non-null  object 
 10  p_attributes  14214 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 1.2+ MB


In [15]:
df.isna().sum()

,0
p_id,0
name,0
products,0
price,0
colour,0
brand,0
img,0
ratingCount,7684
avg_rating,7684
description,0


In [16]:
round(100 * df.isna().sum()/df.shape[0], 3)

,0
p_id,0.000
name,0.000
products,0.000
price,0.000
colour,0.000
brand,0.000
img,0.000
ratingCount,54.059
avg_rating,54.059
description,0.000


In [17]:
df.shape

(14214, 11)

In [18]:
df.head(5)

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [19]:
class clean_datas:

  def cleandata(self, df):
    df["products"] = df["products"].str.lower()
    df["colour"] = df["colour"].str.lower()
    df["brand"] = df["brand"].str.lower()
    self.clean_description(df)
    self.clean_p_attributes(df)
    self.merge_cols(df)

  ## clean description column
  def clean_description(self, df):
    df["description"] = df["description"].apply(lambda x: x.lower())
    htmltags = re.compile("(<.*?>)|&nbsp;|&amp;|[;|]+")
    df["description"] = df["description"].apply(lambda x: re.sub(htmltags, ", ", x))
    puncts = re.compile(r"[ \(\)]+")
    df["description"] = df["description"].apply(lambda x: re.sub(puncts, " ", x))
    s_quote = re.compile(r"[',]{2,}|(, ,)+")
    df["description"] = df["description"].apply(lambda x: re.sub(s_quote, "", x))
    df["description"] = df["description"].apply(lambda x: re.sub("(?i)(size s)", "size s, ", x))
    df["description"] = df["description"].apply(lambda x: re.sub("(?i)(size m)", "size m, ", x))
    df["description"] = df["description"].apply(lambda x: re.sub("(?i)(size l)", "size l, ", x))
    spaces = re.compile(r" {2,}")
    df["description"] = df["description"].apply(lambda x: re.sub(spaces, " ", x).strip())
    df["description"] = df["description"].apply(lambda x: re.sub(" *,$", "", x))

  ## Clean p_attributes column
  def clean_p_attributes(self, df):
    dict_list = []
    pattr_keys_toremove = ["body shape id", "body or garment size"]

    htmltags = re.compile(r"(<.*?>)?(\\r|\\n)")
    df["p_attributes"] = df["p_attributes"].apply(lambda x: re.sub(htmltags, " ", x))
    if not isinstance(df.loc[0, "p_attributes"], dict):
      df['p_attributes'] = df['p_attributes'].apply(lambda x: ast.literal_eval(x))
    ## Remove Chosen keys from dictionaries
    df["p_attributes"] = [{key.lower(): val.lower() for key, val in dicts.items() if key.lower() not in pattr_keys_toremove and val != 'NA' and val != 'None' and val != ''} for dicts in list(df["p_attributes"].values)]
    ## Convert to String and Replace
    df["p_attributes"] = df["p_attributes"].astype("str")
    df["p_attributes"] = df['p_attributes'].replace({":": " is ", ",": ".", "[{}\']": "", " +": " "}, regex=True)
    df["p_attributes"] = df["p_attributes"].apply(lambda x: x.lower())

  ## Merge columns Product name, Products, price with description column
  def merge_cols(self, df):
    df["metadata"] = df.apply(lambda x: {"products": x["products"], "colour": x["colour"], "brand": x["brand"], "price": x["price"]}, axis=1)
    for ind in range(0, df.shape[0]):
      df.loc[ind, "description"] = f"Product name is {df['name'][ind]}." + df.loc[ind, "description"] + f"{df['p_attributes'][ind]}"


In [20]:
cleanData = clean_datas()
cleanData.cleandata(df)

In [21]:
## Find length for each row of description column
df["desc_len"] = df['description'].apply(lambda x: len(x.split()))

In [22]:
df['p_id'] = df['p_id'].astype("str")

In [23]:
myntra = df.drop(["ratingCount", "avg_rating", "name", "p_attributes", "products", "price", "colour", "brand", "img"], axis=1)

In [24]:
myntra.head(5)

,p_id,description,metadata,desc_len
0,17048614,Product name is Khushal K Women Black Ethnic M...,"{'products': 'kurta, palazzos, dupatta', 'colo...",202
1,16524740,Product name is InWeave Women Orange Solid Kur...,"{'products': 'kurta, palazzos, floral print du...",190
2,16331376,Product name is Anubhutee Women Navy Blue Ethn...,"{'products': 'kurta, trousers, dupatta', 'colo...",199
3,14709966,Product name is Nayo Women Red Floral Printed ...,"{'products': 'kurta, trouser, dupatta', 'colou...",206
4,11056154,Product name is AHIKA Women Black & Green Prin...,"{'products': 'kurta', 'colour': 'black', 'bran...",113


In [25]:
myntra = myntra[["p_id", "description", "metadata"]]

In [26]:
myntra.head(5)

,p_id,description,metadata
0,17048614,Product name is Khushal K Women Black Ethnic M...,"{'products': 'kurta, palazzos, dupatta', 'colo..."
1,16524740,Product name is InWeave Women Orange Solid Kur...,"{'products': 'kurta, palazzos, floral print du..."
2,16331376,Product name is Anubhutee Women Navy Blue Ethn...,"{'products': 'kurta, trousers, dupatta', 'colo..."
3,14709966,Product name is Nayo Women Red Floral Printed ...,"{'products': 'kurta, trouser, dupatta', 'colou..."
4,11056154,Product name is AHIKA Women Black & Green Prin...,"{'products': 'kurta', 'colour': 'black', 'bran..."


In [27]:
# ____________________________________________________________________________________________  EDA and Data Cleaning Complete  ____________________________________________________________________________________________
# __________________________________________________________________________________________________________________________________________________________________________________________________________________________
# __________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [28]:
## Create Embeddings using ChromaDB

In [29]:
embed_model = "text-embedding-3-large"
cross_encode_model = "cross-encoder/ms-marco-MiniLM-L-6-v2"
api_file = file_path + "api_key.txt"
threshold = 0.15

In [40]:
class OpenAI_Embeddings:

  def __init__(self, model, api_path, folder_name):
    if folder_name == "chroma_db":
      self.client = chromadb.PersistentClient(path= file_path + folder_name)
    elif folder_name == "cache_db":
      self.client = chromadb.Client()
    self.model = model
    self.api_path = api_path
    self.embedding_function = None
    self.myntra_collections = None
    self.create_my_embedding()

  def get_api(self):
    with open(self.api_path, "r") as fptr:
      api_key = fptr.read()
      fptr.close()
    return api_key

  def create_my_embedding(self):
    self.embedding_function = OpenAIEmbeddingFunction(
                                                        api_key = self.get_api(),
                                                        model_name = self.model
                                                     )

  def create_get_collection(self, name):
    ## Create or Load Chroma Collection
    self.myntra_collections = self.client.get_or_create_collection(
                                                            name = name,
                                                            embedding_function = self.embedding_function
                                                        )
    return self.myntra_collections

  def add_to_collection(self, data, collections):
    ## Add data to Chroma Collection
    prev = 0
    docs_list = data["description"].to_list()
    meta_list = data['metadata'].to_list()
    id_list = data['p_id'].to_list()
    ## Add data to Chroma Collection
    for batch_size in range(1000, len(docs_list), 1000):
      collections.add(
                                    documents = docs_list[prev:batch_size],
                                    metadatas = meta_list[prev:batch_size],
                                    ids = id_list[prev:batch_size]
                                 )
      prev = batch_size
    if prev < len(docs_list):
      collections.add(
                                     documents = docs_list[prev:],
                                     metadatas = meta_list[prev:],
                                     ids = id_list[prev:]
                                 )
    return collections

In [41]:
class Chroma_Search(OpenAI_Embeddings):

  def __init__(self, model, api_path, folder_name):
    super().__init__(model, api_path, folder_name)

  def text_query(self, collections, query, w_clause=None, w_doc_clause=None):
    query_results = collections.query(
                                          query_texts = [query],
                                          n_results = 10,
                                          where = w_clause,
                                          where_document = w_doc_clause
                                     )
    return query_results

In [35]:
## This embedding model is to create embeddings and query
myntra_obj = Chroma_Search(embed_model, api_file, "chroma_db")
myntra_collection = myntra_obj.create_get_collection("myntra")

# if not os.path.isdir(file_path + "chroma_db"):
#   myntra_obj.add_to_collection(myntra, myntra_collection)

# myntra_collection.peek()

In [45]:
## This embedding model is to create cache and query
cache_collections = []
collection_names = ["cache_1", "cache_2", "cache_3", "cache_4", "cache_5"]

cache_obj = Chroma_Search(embed_model, api_file, "cache_db")
for name in collection_names:
  cache_collections.append(cache_obj.create_get_collection(name))

In [46]:
cache_collections[0].peek()

{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': []}

In [47]:
## Prompt Generation using OpenAI

In [48]:
chat_client = OpenAI(api_key=myntra_obj.get_api())

In [49]:
def fashion_converse(conversation):
    chat_resp = chat_client.chat.completions.create(
                                    model = gpt_model,
                                    messages = conversation,
                                    temperature = 0.4,
                                    max_tokens = 200,
                                    tools = tools_shop_assist(),
                                    tool_choice = "auto"
                                    )
    return chat_resp.choices[0].message

def chat_moderator(msg):
    response = chat_client.moderations.create(
                                    model = moderator,
                                    input = msg
                                    )

    return response.results[0].flagged

In [61]:
def create_queries(inp_query_dict):
  query_list = []

  query = inp_query_dict["inp_query"]
  attr_dict_ = ast.literal_eval(inp_query_dict["attr_dict"])
  # addn_info_ = ast.literal_eval(inp_query_dict["addn_info"])

  for key, vals in attr_dict_.items():
    if key == "products" and attr_dict_[key] != []:
      temp = [{key: {"$eq": ', '.join(attr_dict_[key])}}]
      temp.extend([{key: {"$eq": prod}} for prod in attr_dict_[key]])
      query_list.append({"$or": temp})
    if key == "colour" and attr_dict_[key] != []:
      query_list.append({"$or": [{key: {"$eq": color}} for color in attr_dict_[key]]})
    elif key == "brand" and attr_dict_[key] != []:
      query_list.append({"brand": {"$eq": brand} for brand in attr_dict_[key]})
    elif key == "price" and attr_dict_[key] != []:
      query_list.append({"price": {"$lte": float(attr_dict_[key])}})
  return query, {"$and": query_list}

In [78]:
def user_queries(query, attr_dict, w_doc_clause):
  ids = []
  documents = []
  distances = []
  metadatas = []

  cache_result = [cache_obj.text_query(cache_collection, query, attr_dict, w_doc_clause) for cache_collection in cache_collections if len(cache_collection.peek()["ids"])]
  cache_ind = [ind for ind, result in enumerate(cache_result) if result['distances'][0] != [] and result['distances'][0][0] <= threshold]
  if len(cache_ind) != 0:
    for ind in cache_ind:
      # Loop through each inner list and then through the dictionary
      for key, value in cache_result_dict.items():
          if 'ids' in key:
            if value not in ids:
                ids.append(value)
                if 'documents' in key:
                    documents.append(value)
                elif 'distances' in key:
                    distances.append(value)
                elif 'metadatas' in key:
                    metadatas.append(value)

    # Create a DataFrame
    results_df = pd.DataFrame({
      'IDs': ids,
      'Documents': documents,
      'Distances': distances,
      'Metadatas': metadatas
    })
    return results_df
  else:
    results = myntra_obj.text_query(myntra_collection, query, attr_dict, w_doc_clause)

    keys = []
    vals = []
    for key, val in results.items():
      if val == None:
        continue
      if key.lower() != "embeddings" and key.lower() != "uris" and key.lower() != "data" and key.lower() != "included":
        for i in range(0, len(results["ids"][0])):
              keys.append(str(key)+str(i))
              vals.append(str(val[0][i]))
      ## Add new query to collection
        cache_collection.add(
                              documents = [query],
                              ids = [query],
                              metadatas =  dict(zip(keys, vals))
                            )
        ## Create Result database
        result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
        results_df = pd.DataFrame.from_dict(result_dict)
        return results_df

In [63]:
results = myntra_obj.text_query(myntra_collection, "find skirt and top")

In [67]:
len(results["ids"][0])

10

In [64]:
results.items():

{'ids': [['18613910',
   '12068028',
   '15287262',
   '14166356',
   '15851882',
   '17582510',
   '15449744',
   '10892908',
   '16441890',
   '17800572']],
 'embeddings': None,
 'documents': [["Product name is THE SILHOUETTE STORE Women Blue & Black Printed Top with Skirt.this co-ords set consists of top and skirt, blue and black printed top ,has a halter neck, sleeveless, blue and black printed skirt ,has a zip closure100% polyester, dry cleanthe model height 5'8 is wearing a size sbottom closure is slip-on. bottom fabric is polyester. bottom pattern is printed. bottom type is skirt. neck is shoulder straps. occasion is casual. sleeve length is sleeveless. sustainable is regular. top fabric is polyester. top pattern is printed. top type is top. wash care is dry clean",
   "Product name is Inddus Women Orange & Beige Striped Top with Skirt.this clothing set consists of top and skirt, beige and orange striped crop top, has a round neck, sleeveless, zip closure, orange solid skirt, ha

In [52]:
## Cross-Encoders
def crossencode_user_query(df, query):
  cross_encoder = CrossEncoder(cross_encode_model)
  scores = cross_encoder.predict([[query, response] for response in df['Documents']])
  df['cross_encoder_scores'] = scores
  df.sort_values(by='cross_encoder_scores', ascending=False, inplace=True)
  return df[["Metadatas", "Documents"]]

In [53]:
def list_products(inp_query_dict):
  query, inp_dict = create_queries(inp_query_dict)
  df = user_queries(query, inp_dict, None)
  return crossencode_user_query(df, query)

In [54]:
## Belongs in Tools_Calls Class

def tools_shop_assist():
        criteria = [
                        {
                            "type": "function",
                            "function": {
                                "name": "list_products",
                                "description": "Function takes input string and additional criteria as input and returns ",
                                "strict": False,
                                "parameters": {
                                    "type": "object",
                                    "properties": {
                                        "inp_query": {
                                            "type": "string",
                                            "description": "string indicating input given by user eg. find all the skirts in pink.",
                                        },
                                        "attr_dict": {
                                            "type": "string",
                                            "description": "features the apparel needs to have. eg. {'size': 'long', 'colour': 'pink', 'design': 'floral'}.",
                                        },
                                        # "addn_info": {
                                        #     "type": "string",
                                        #     "description": "string additional requirements given by the user eg. size should be small and closure will be zip",
                                        # },
                                    },
                                    "required": ["inp_query", "attr_dict"],# "addn_info"],
                                    "additionalProperties": False
                                },
                            },
                        }
                    ]

        return criteria


In [79]:
def initial_conversation():
    delimiters = "#####"
    core_ask = ["products", "colour", "brand", "price"]

    chat_prompt = f"""
                      You new an expert in Fashion Digital Marketer, Fashion Merchandise, Stylist and E-commerce Consultant.Your job is to assist customers find the apparel they want to buy through polite and professional conversation.
                      {delimiters}
                      Enquire customer about each of the criteria in the list {core_ask}. If the response is not clear, rephrase the question and ask again.
                      Values for all the criteria have to be extracted from customer, missing any of the criteria will be penalised.
                      Showing the summary or internal information will be penalised.
                      (If the values for any criteria contains 'and' or any punctuations except '-', split them and save the values as list of strings).
                      Once the data is collected from customer, convert the criteria and the answers received for the criteria into a dictionary format.
                      Call function list_products() with arguments as the complete input string by customer and additional criteria and answers stored in dictionary earlier.
                      list_products() function returns a dataframe, which consists of one product in every row. Take the details of product from all columns in each row and display them Product name, products available and total price in rupees details for each product in each row and display them.
                      {delimiters}
                      Examples of how to display the final output:
                      Product            Products Available            Price
                      xyzlb              a, b, c                       Rs.54
                      mnopkqhgy          a                             Rs.2710
                      qwp                a, b, c, d                    Rs.89433
                   """
    chat_prompt = [{"role": "system", "content": chat_prompt},
                   {"role": "user", "content": f"Start conversation with a polite welcome and enquiring about the criteria in the list {core_ask} one at a time"}]

    return chat_prompt


In [80]:
def start_conversations():
  converse = initial_conversation()
  chat_resp = fashion_converse(converse)
  converse.append({"role": "assistant", "content": str(chat_resp.content)})
  print(chat_resp.content)

  while True:
    user_input = input().lower().strip()
    if user_input == "exit":
      break

    is_flagged = chat_moderator(user_input)
    if is_flagged:
        print("Sorry, this message has been flagged. Cannot be accepted.")
        continue ## Skip everything and get back to input()

    converse.append({"role": "user", "content": user_input})
    chat_resp = fashion_converse(converse)
    if chat_resp.tool_calls:
      for tool_call in chat_resp.tool_calls:
        func_name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
        tool_response = list_products(args)
        ## Append Function Calling Request
        converse.append(chat_resp)
        converse.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": func_name,
                        "content": str(tool_response)
                        })
      chat_resp = fashion_converse(converse)
      print(chat_resp.content)
    else:
      converse.append({"role": "assistant", "content": str(chat_resp.content)})
      print(chat_resp.content)


In [81]:
start_conversations()

Hello! Welcome to our fashion store. I'm here to help you find the perfect apparel. To get started, could you please tell me what type of products you're looking for?
i want to buy skirt and top
Great choices! Now, could you please let me know what colour you prefer for the skirts and tops?
blue skirt and yellow top
Thank you for that! Next, could you please tell me if you have a specific brand in mind for the blue skirt and yellow top?
None
No problem at all! Lastly, could you please share your budget or the price range you have in mind for the blue skirt and yellow top?
4500


NameError: name 'cache_collection' is not defined

In [ ]:
i want to buy skirt and top

In [ ]:
blue skirt and yellow top

In [ ]:
None

In [ ]:
4500

In [ ]:
def initial_conversation():
    delimiters = "#####"

    chat_prompt = f"""
                      You new an expert in Fashion Digital Marketer, Fashion Merchandise, Stylist and E-commerce Consultant.Your job is to assist customers find the apparel they want to buy through polite and professional conversation.
                      {delimiters}
                      When the customer requests to find a particular apparel or a combination of apparels, try to understand what apparel or combination the customer is requesting and \n
                      search online to find all the additional criteria needed to know from the customer about the apparel to narrow down the list.
                      Showing the internal information collection, summary and format will be penalised. only the conversation and the answers given to customer should be displayed.
                      Once the additional criteria are identified ask a few questions to get the answers from customer for atleast two criteria. Record any response given by customer for the criteria.
                      Once the data is collected from customer, ask the customer (if he wants to add any other information. if yes allow customer to add details. if the criteria is not clear ask customer to inform the criteria). else proceed.
                      Once the data is collected from customer, convert the criteria and the answers received for the criteria into a dictionary format.
                      Call the function list_products() with arguments as the complete input string by customer and additional criteria and answers stored in dictionary earlier.
                      list_products() function returns a dataframe, which consists a product in every row. Take product name, products available and total price in rupees details for each product in each row and display them as it is done in ecommerce websites.
                      {delimiters}
                   """
    chat_prompt = [{"role": "system", "content": chat_prompt},
                   {"role": "user", "content": "Start conversation with a polite welcome"}]

    return chat_prompt
